In [3]:
import pandas as pd
#from google.colab import files
#uploaded = files.upload()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['ff.csv']))
df.head()

,shop,product_name,comments,quality,catagories
0,daraz,Original i7s TWS Wireless Bluetooth AirPods Ea...,প্রোডাক্টটি অনেক সুন্দর ডেলিভারি অনেক আগে পেয়...,good,p
1,daraz,Water proof Mini Submersible 12 Volt 8 Watt DC...,দাম অনেক বেশি দোকানে অনেক কমদামে পাওয়া যায় ।...,bad,c
2,daraz,3000 mAh Rechargable Power Bank,"ভালই আছে খারাপ নয়, আমি চালাচ্ছি ভালই চলছে তবে ...",good,c
3,daraz,Unstitched Black High Quality Cotton Fabric Ex...,faltu ekta jama👊.vuleo keo jama kinben na dara...,bad,c
4,daraz,Havit HV-H2116D Stereo Headset with PC Microph...,চমৎকার একটি প্রোডাক্ট। খুবই ভালো প্রোডাক্ট। বে...,good,p


In [4]:
df=pd.read_excel('/content/drive/MyDrive/Munna_Research/Experiment/ecdata_new.xlsx')
df

,shop,product_name,comments,quality,catagories
0,daraz,MyCell Mi10 PLUS - 2.4 Feature Phone,Recommended but phone battery cover is not as ...,bad,a
1,daraz,Mini Hand Sanitizer Anti-Bacterial Hand Gel 30...,thanks a lot but what is the problem you guys ...,bad,a
2,daraz,"Vietnam hybrids coconut Tree , ১ পিস গাছের চারা",মাত্র ১১ দিনে ডেলিভারি পেয়েছি সে জন্য দারাজকে ...,bad,a
3,daraz,Bogesi Men Gold Artificial Leather Wallet,bogesi অরজিনাল প্রোডাক্টই পেয়েছি। ধন্যবাদ সেলর,bad,a
4,daraz,Craft HD LED TV - 32(Inch),Avarege...I Faced Some Problems But Seller And...,bad,a
...,...,...,...,...,...
5104,daraz,Black and White Cotton Un-Stitched Salwar Kame...,জামাটা ঠিক আছে কিন্তু অলনা টা মিলে নাই ইমেজে দ...,good,w
5105,daraz,Black and White Cotton Un-Stitched Salwar Kame...,জামাটা ঠিক আছে কিন্তু অলনা টা মিলে নাই ইমেজে দ...,good,w
5106,daraz,Both Sided Gum Tape - 2 inch - White\n \n \n...,orginal prodauct and draz delevery good prodau...,good,w
5107,daraz,2020 NEW STYLE Cotton Long Sleeve Formal Shirt...,Order r real shirt check koren\n\n\n\n\npleaz ...,good,w


In [5]:
df=df.drop(['product_name','shop','quality'],axis=1)

In [6]:
df.catagories.unique()

array(['a', 'c', 'r', 'w'], dtype=object)

In [7]:
df['catagories'].replace({'c':'complain','w':'wrong delevery','a':'appreciation','r':'recommended'}, inplace=True)
df.head()

,comments,catagories
0,Recommended but phone battery cover is not as ...,appreciation
1,thanks a lot but what is the problem you guys ...,appreciation
2,মাত্র ১১ দিনে ডেলিভারি পেয়েছি সে জন্য দারাজকে ...,appreciation
3,bogesi অরজিনাল প্রোডাক্টই পেয়েছি। ধন্যবাদ সেলর,appreciation
4,Avarege...I Faced Some Problems But Seller And...,appreciation


In [8]:
df.catagories.unique()

array(['appreciation', 'complain', 'recommended', 'wrong delevery'],
      dtype=object)

In [9]:
df=df.dropna()
df

,comments,catagories
0,Recommended but phone battery cover is not as ...,appreciation
1,thanks a lot but what is the problem you guys ...,appreciation
2,মাত্র ১১ দিনে ডেলিভারি পেয়েছি সে জন্য দারাজকে ...,appreciation
3,bogesi অরজিনাল প্রোডাক্টই পেয়েছি। ধন্যবাদ সেলর,appreciation
4,Avarege...I Faced Some Problems But Seller And...,appreciation
...,...,...
5104,জামাটা ঠিক আছে কিন্তু অলনা টা মিলে নাই ইমেজে দ...,wrong delevery
5105,জামাটা ঠিক আছে কিন্তু অলনা টা মিলে নাই ইমেজে দ...,wrong delevery
5106,orginal prodauct and draz delevery good prodau...,wrong delevery
5107,Order r real shirt check koren\n\n\n\n\npleaz ...,wrong delevery


In [10]:
from nltk.stem.porter import PorterStemmer

porter=PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [11]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text
df['comments']=df['comments'].apply(preprocessor)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(strip_accents= None,
                    lowercase=False,
                    preprocessor=None,
                    tokenizer=tokenizer_porter,
                    norm='l2',
                    smooth_idf=True )
y=df.catagories.values
X=tfidf.fit_transform(df.comments)


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state= 44, test_size=0.2)
                                                   

LOGISTIC REGRATION



In [14]:
from sklearn.linear_model import LogisticRegression
lc = LogisticRegression()
lc.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
pred=lc.predict(X_test)

In [25]:
print(classification_report(y_test,pred))

                precision    recall  f1-score   support

  appreciation       0.92      0.65      0.76       231
      complain       0.73      0.92      0.82       381
   recommended       0.78      0.79      0.78       332
wrong delevery       0.78      0.46      0.58        78

      accuracy                           0.78      1022
     macro avg       0.80      0.70      0.74      1022
  weighted avg       0.79      0.78      0.78      1022



In [29]:
mat = confusion_matrix(pred, y_test)
pd.DataFrame(mat)

,0,1,2,3
0,150,1,12,0
1,35,351,55,37
2,44,25,261,5
3,2,4,4,36


KNN


In [38]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=6)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [40]:
pred=neigh.predict(X_test)
print(classification_report(y_test,pred))
mat = confusion_matrix(pred, y_test)
pd.DataFrame(mat)

                precision    recall  f1-score   support

  appreciation       0.63      0.57      0.60       231
      complain       0.77      0.72      0.74       381
   recommended       0.62      0.72      0.67       332
wrong delevery       0.65      0.59      0.62        78

      accuracy                           0.68      1022
     macro avg       0.67      0.65      0.66      1022
  weighted avg       0.68      0.68      0.68      1022



,0,1,2,3
0,131,21,49,6
1,25,275,40,19
2,70,68,240,7
3,5,17,3,46
